In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
from keras import layers,optimizers,losses

physical_devices = tf.config.list_physical_devices("gPU")

# Data preparation

In [ ]:

(ds_train, _),infos=tfds.load(
    name="mnist",
    split=["train","test"],
    shuffle_files=True,
    with_info=True ,  
    as_supervised=False
)

def normalize_to_normal(x):
    return 255*(x-1)/2

def normalize_data(data):
    return tf.cast(2*(data["image"]/255)-1,tf.float32)

ds_train=ds_train.map(normalize_data,num_parallel_calls=tf.data.AUTOTUNE)
ds_train=ds_train.cache()
ds_train=ds_train.shuffle(infos.splits["train"].num_examples)#60,000
ds_train=ds_train.batch(batch_size=64)
ds_train=ds_train.prefetch(tf.data.AUTOTUNE)

print(ds_train.take(0))

## Build Model

### Generator Network

In [ ]:
class CNNTranspose(layers.Layer):
    def __init__(self,channels,filter_size,stride=2,padding="same",name=None):
        super(CNNTranspose,self).__init__(name=name)
        self.cnt=layers.Conv2DTranspose(channels,filter_size,stride,padding,use_bias=False)
        self.bn=layers.BatchNormalization()
    def call(self,inputs,training=False):
        x=self.cnt(inputs)
        x=self.bn(x,training=training)
        return tf.nn.relu(x)



class Generator(keras.Model):
    seed_dim=128
    def __init__(self):
        super(Generator,self).__init__()
        self.linear=layers.Dense(7*7*256,name="linear")
        self.tcn1=CNNTranspose(128,5,stride=1,name='TPCNN_1')
        self.tcn2=CNNTranspose(64,5,stride=2,name="TPCNN_2")
        self.output_layer=layers.Conv2DTranspose(1,3,2,"same",name='output')
    def call(self,inputs,training=False):
        x=self.linear(inputs)
        x=layers.Reshape((7,7,256))(x)
        x=self.tcn1(x,training=training)
        x=self.tcn2(x,training=training)
        x=self.output_layer(x)
        return tf.keras.activations.tanh(x)
    def architecture(self):
        x=keras.Input((128))
        model=tf.keras.Model(inputs=[x],outputs=self.call(x))
        return model.summary()
    
generator=Generator()
generator.architecture()

### Discriminator Network

In [ ]:
class CNN(layers.Layer):
    def __init__(self,channels,filter_size,name=None):
        super(CNN,self).__init__(name=name)
        self.cnn=layers.Conv2D(channels,filter_size,strides=2,padding="same",use_bias=False)
        self.bn=layers.BatchNormalization()
    def call(self,inputs,training=False):
        x=self.cnn(inputs)
        x=self.bn(x,training=training)
        return tf.nn.leaky_relu(x)


class Discriminator(keras.Model):
    def __init__(self):
        super(Discriminator,self).__init__()
        self.cnn1=layers.Conv2D(64,3,2,padding="same",name="Conv_1")
        self.cnn2=CNN(128,5,name="Conv_2")
        self.cnn3=CNN(256,5,name="Conv_1")
        self.output_layer=layers.Dense(1,'sigmoid',name="output")
    def call(self,inputs,training=False):
        x=self.cnn1(inputs)
        x=tf.nn.leaky_relu(x)
        x=self.cnn2(x,training=training)
        #x=self.cnn3(x,training=training)
        x=layers.Flatten()(x)
        x=self.output_layer(x)
        return x
    def architecture(self):
        x=keras.Input((28,28,1))
        model=tf.keras.Model(inputs=[x],outputs=self.call(x))
        return model.summary()


discriminator=Discriminator()
discriminator.architecture()

# Train the Networks


### Optimizer and Loss Function

* loss function : log(D(x))+log(1-D(G(z)))
* Optimizer : Adam
* learning rate :3e-4


In [ ]:
gen_opt=optimizers.Adam(1e-4)
disc_opt=optimizers.Adam(1e-4)
loss_fn=losses.BinaryCrossentropy()
num_epochs=20

### Training loop
* the Discriminator try to maximize the loss function
* the Generator try to minimize log(1-D(G(Z))) or maximize log(D(G(x)))

In [ ]:
import tqdm
import os

for epoch in range(num_epochs):
    for idx,real in enumerate(tqdm.tqdm(ds_train)):
        bach_size=real.shape[0]
        z=tf.random.normal([bach_size,Generator.seed_dim])  #random noise
        
        
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape : 
            fake=generator(z,training=True) #G(z)

            real_loss=loss_fn(tf.ones(bach_size,1),discriminator(real,training=True)) #log(D(x)) 
            fake_loss=loss_fn(tf.zeros(bach_size,1),discriminator(fake,training=True))#log(1-D(G(Z)))

            disc_loss=real_loss+fake_loss
            gen_loss=loss_fn(tf.ones(bach_size,1),discriminator(fake))
        
        grads_gen=gen_tape.gradient(gen_loss,generator.trainable_variables)
        gen_opt.apply_gradients(zip(grads_gen,generator.trainable_variables))
        grads_disc=disc_tape.gradient(disc_loss,discriminator.trainable_variables)
        disc_opt.apply_gradients(zip(grads_disc,discriminator.trainable_variables))
        generated_image=generator(z,training=False)
        if idx % 100 ==0:
            img=tf.keras.preprocessing.image.array_to_img(normalize_to_normal(generated_image[0]))
            img.save(f"generated_images/generated_img{epoch}_{idx}_.png")